#  4_2_03_Beneficios privados hacia parientes del declarante provenientes de sectores con posible conflicto de interés
| **Escenario**                                                                  | **Condición exacta evaluada**                                                                                           | **Resultado**    |
| ------------------------------------------------------------------------------ | ----------------------------------------------------------------------------------------------------------------------- | ---------------- |
| **No hay beneficios privados declarados hacia parientes**                      | La lista `beneficiosPrivados.beneficio` no existe o está vacía                                                          | ⚪ **NO_APLICA**  |
| **Faltan datos esenciales**                                                    | No existe:  <br>• nombre del ente público, o <br>• sector del beneficio, o <br>• la relación de beneficiario (pariente) | ⚪ **SIN_DATO**   |
| **El beneficio hacia un pariente proviene de un sector marcado como T, D o C** | `sector ∈ sectores_riesgo[funcion_principal_del_ente]`                                                                  | 🔴 **NO_CUMPLE** |
| **El beneficio NO proviene de sectores T, D o C**                              | `sector` **no** se encuentra en los sectores riesgo asociados a la función principal del ente público                   | 🟢 **CUMPLE**    |
| **La lista de beneficios existe, pero los campos están vacíos o nulos**        | Uno o más beneficios carecen de sector o beneficiario                                                                   | ⚪ **SIN_DATO**   |


In [1]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME
import unicodedata


METRIC_ID = "4_2_03_SECTOR_RIESGO_BENEFICIOS_PARIENTES"


# -------------------------------------------------------------------
# MATRIZ TEMPORAL (reemplazar por tu catálogo oficial INEGI)
# -------------------------------------------------------------------
# FUNCION PRINCIPAL DEL ENTE -> SECTORES RIESGO (T, D, C)
MATRIZ_SECTORES_FUNCIONES = {
    # Ejemplos — reemplazar por catálogo oficial
    "SALUD": ["T", "D"],
    "OBRAS PUBLICAS": ["T", "C"],
    "EDUCACION": ["D"],
    "SEGURIDAD": ["T", "C"],
}


# -------------------------------------------------------------------
# AUXILIARES
# -------------------------------------------------------------------
def limpiar(v):
    if not v:
        return None
    v = str(v).strip().upper()
    # quitar acentos
    v = ''.join(c for c in unicodedata.normalize('NFD', v)
                if unicodedata.category(c) != 'Mn')
    # quitar dobles espacios
    while "  " in v:
        v = v.replace("  ", " ")
    return v


def extraer(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None
        return actual
    except:
        return None


# -------------------------------------------------------------------
# EVALUADOR DE LA MÉTRICA
# -------------------------------------------------------------------
def evaluar_metrica(doc):

    # 🅐 Nombre del ente público
    ente = limpiar(
        extraer(doc, "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nombreEntePublico")
    )

    if not ente:
        return "SIN_DATO"

    # Aquí deberías mapear el ente → función principal real
    funcion_principal = ente  # 🔴 Temporal hasta tener tu catálogo oficial

    sectores_riesgo = MATRIZ_SECTORES_FUNCIONES.get(funcion_principal, [])

    # 🅑 Beneficios privados declarados
    beneficios = extraer(
        doc, "declaracion.intereses.beneficiosPrivados.beneficio"
    )

    if not beneficios:
        return "NO_APLICA"

    if not isinstance(beneficios, list):
        beneficios = [beneficios]

    hubo_datos = False

    for ben in beneficios:

        sector = limpiar(ben.get("sector"))
        beneficiario = limpiar(ben.get("beneficiario"))

        # *** Aquí está el cambio respecto a la métrica 4_2_02 ***
        # VALIDAR QUE EL BENEFICIARIO SEA UN PARIENTE
        # Recuerda: tú defines tu lista de parientes válidos
        # Ejemplo simplificado:
        PARIENTES = [
            "PADRE", "MADRE", "HERMANO", "HERMANA", "ESPOSO", "ESPOSA",
            "HIJO", "HIJA", "PAREJA", "CONCUBINA", "CONCUBINARIO"
        ]

        if beneficiario not in PARIENTES:
            # no es beneficio de pariente -> no se evalúa
            continue

        if not sector:
            return "SIN_DATO"

        hubo_datos = True

        # Validación principal
        if sector in sectores_riesgo:
            return "NO_CUMPLE"

    if not hubo_datos:
        return "NO_APLICA"

    return "CUMPLE"


# -------------------------------------------------------------------
# PROCESADOR MONGO
# -------------------------------------------------------------------
def procesar_metrica_4_2_03():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"\n🚀 Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]

        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosEmpleoCargoComision": 1,
            "declaracion.intereses.beneficiosPrivados": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"⚠️ Error en documento {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f" > {total} documentos procesados...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print("Total procesados:", total)
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass

        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_2_03()



🚀 Procesando métrica 4_2_03_SECTOR_RIESGO_BENEFICIOS_PARIENTES...



c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


 > 2000 documentos procesados...
 > 4000 documentos procesados...
 > 6000 documentos procesados...
 > 8000 documentos procesados...
 > 10000 documentos procesados...
 > 12000 documentos procesados...
 > 14000 documentos procesados...
 > 16000 documentos procesados...
Conexión cerrada.


Traceback (most recent call last):
  File "C:\Users\Mauro\AppData\Local\Temp\ipykernel_12032\718103134.py", line 142, in procesar_metrica_4_2_03
    for doc in cursor:
               ^^^^^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\cursor.py", line 1289, in __next__
    return self.next()
           ~~~~~~~~~^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\cursor.py", line 1265, in next
    if len(self._data) or self._refresh():
                          ~~~~~~~~~~~~~^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\cursor.py", line 1236, in _refresh
    self._send_message(g)
    ~~~~~~~~~~~~~~~~~~^^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\cursor.py", line 1108, in _send_message
    response = client._run_operation(
        operation, self._unpack_response, address=self._address
    )
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\_csot.py", line 125, in csot_wrappe